# Legal-RAG 检索 Benchmark（合同法示例）

本 Notebook 对比几种不同的检索配置在 **《民法典·合同编》** 场景下的效果：

1. **BM25 only**：纯文本倒排检索（lexical）
2. **Dense only**：BGE-base-zh-v1.5 + FAISS（纯语义向量）
3. **Hybrid**：dense + BM25 加权融合
4. **Graph-augmented**：在 Hybrid 基础上叠加 law_graph 扩展 + 语义 rerank 

指标：
- Hit@1 / Hit@3 / Hit@5 / Hit@10

> 说明：
> - 本 Notebook 只评估“检索命中情况”，**不调用 LLM**。
> - Ground truth 存在于 `data/eval/contract_law_qa.jsonl`，如果不存在，会自动写入一份简单示例。

## 1. 环境与路径检查

需要提前运行：

```bash
python -m scripts.preprocess_law
python -m scripts.build_index
```

已经构建好：
- `data/processed/contract_law.jsonl`
- `data/index/faiss.index` / `data/index/faiss_meta.jsonl`
- `data/index/bm25.pkl`

In [ ]:
from pathlib import Path
import json
from typing import List, Dict, Any

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from legalrag.config import AppConfig
from legalrag.models import RetrievalHit, QueryType, RoutingMode, LawChunk
from legalrag.retrieval.bm25_retriever import BM25Retriever
from legalrag.retrieval.vector_store import VectorStore
from legalrag.retrieval.hybrid_retriever import HybridRetriever
from legalrag.pipeline.rag_pipeline import RagPipeline
from legalrag.routing.router import QueryRouter
from legalrag.utils.logger import get_logger

logger = get_logger(__name__)

cfg = AppConfig.load()
BASE_DIR = Path(cfg.paths.base_dir)
DATA_DIR = Path(cfg.paths.data_dir)
EVAL_DIR = Path(cfg.paths.eval_dir)
EVAL_DIR.mkdir(parents=True, exist_ok=True)

print("BASE_DIR:", BASE_DIR)
print("DATA_DIR:", DATA_DIR)
print("EVAL_DIR:", EVAL_DIR)

## 2. 构造 / 加载 Ground Truth 数据集

期望的 `contract_law_qa.jsonl` 结构示例：

```json
{"question": "合同约定的违约金为合同金额的 40%，是否合理？", "target_articles": ["约定违约金的调整", "第 585 条"]}
{"question": "什么是不可抗力？", "target_articles": ["不可抗力", "第 590 条"]}
```

这里我们做一个简单约定：
- `target_articles` 里既可以是条文编号（如 "第585条"），也可以是条文标题关键词（如 "不可抗力"）
- 在匹配时会同时判断：
  - 检索结果的 `article_no` 是否包含以上任意字符串
  - 或者检索结果全文是否包含这些关键词

In [ ]:
eval_path = EVAL_DIR / "contract_law_qa.jsonl"

if not eval_path.exists():
    print("[INFO] eval file not found, creating a small toy set at:", eval_path)
    toy_data = [
        {
            "question": "合同约定的违约金为合同金额的 40%，是否合理？",
            "target_articles": ["违约金", "过高", "调整"]
        },
        {
            "question": "什么是不可抗力？",
            "target_articles": ["不可抗力"]
        },
        {
            "question": "合同一方迟延履行，另一方可以解除合同的条件是什么？",
            "target_articles": ["解除合同", "迟延履行"]
        },
        {
            "question": "当事人约定了定金，违约时如何处理定金？",
            "target_articles": ["定金", "定金罚则"]
        }
    ]
    with eval_path.open("w", encoding="utf-8") as f:
        for obj in toy_data:
            f.write(json.dumps(obj, ensure_ascii=False) + "\n")


def load_eval_data(path: Path) -> List[Dict[str, Any]]:
    data = []
    with path.open("r", encoding="utf-8") as f:
        for line in f:
            line = line.strip()
            if not line:
                continue
            data.append(json.loads(line))
    return data


eval_data = load_eval_data(eval_path)
print(f"Loaded {len(eval_data)} eval questions from {eval_path}")
eval_data[:2]

## 3. 初始化检索与 Graph-aware 组件

这里我们初始化四种“检索模式”对应的对象：

1. `BM25Retriever` – 纯文本检索
2. `VectorStore` – dense（BGE + FAISS）
3. `HybridRetriever` – dense + sparse
4. `RagPipeline` – 其中的 `_graph_augmented_retrieval` 作为 Graph-aware 模式（不调用 LLM）

> 注意：Graph 模式仍然依赖 Hybrid + law_graph + BGE encoder，属于在前三者基础上的扩展。

In [ ]:
bm25 = BM25Retriever(cfg)
vector_store = VectorStore(cfg)
hybrid = HybridRetriever(cfg)
pipeline = RagPipeline(cfg)
router = QueryRouter(llm_client=pipeline.llm, llm_based=cfg.routing.llm_based)

print("BM25, VectorStore, Hybrid, RagPipeline initialized.")

## 4. Hit@K 评估函数

对于每个问题，我们会计算：

- top-k 检索结果中是否出现“正确条文”（命中计为 1，未命中计为 0）
- 对所有问题取平均，得到 Hit@K

评估逻辑：
- `target_articles` 是一组字符串，可匹配：
  - `hit.chunk.article_no` 字段
  - 或 `hit.chunk.text` 中是否包含这些关键词

这样既兼容“目标是条号”，也兼容“目标是若干关键词／条文标签”。

In [ ]:
from collections import defaultdict
from typing import Callable

def is_hit(chunk: LawChunk, targets: List[str]) -> bool:
    if not targets:
        return False
    text = (chunk.text or "")
    article_no = (getattr(chunk, "article_no", "") or "")
    for t in targets:
        t = t.strip()
        if not t:
            continue
        if t in article_no:
            return True
        if t in text:
            return True
    return False


def eval_retriever(
    name: str,
    retrieve_fn: Callable[[str, int], List[RetrievalHit]],
    eval_data: List[Dict[str, Any]],
    ks: List[int] = [1, 3, 5, 10],
) -> Dict[str, float]:
    """通用 Hit@K 评估函数。

    retrieve_fn: (question, top_k) -> List[RetrievalHit]
    """
    stats = {f"hit@{k}": 0.0 for k in ks}
    n = 0

    for item in eval_data:
        q = item.get("question", "")
        targets = item.get("target_articles", [])
        if not q:
            continue

        max_k = max(ks)
        try:
            hits = retrieve_fn(q, max_k)
        except Exception as e:
            logger.error(f"[{name}] failed on question: {q}; error={e}")
            continue

        n += 1
        for k in ks:
            top_hits = hits[:k]
            if any(is_hit(h.chunk, targets) for h in top_hits):
                stats[f"hit@{k}"] += 1.0

    if n == 0:
        return {f"hit@{k}": 0.0 for k in ks}

    for k in ks:
        stats[f"hit@{k}"] /= n

    print(f"[{name}] evaluated on {n} questions:")
    for k in ks:
        print(f"  Hit@{k}: {stats[f'hit@{k}']:.3f}")

    return stats

## 5. 定义各检索模式的接口

我们分别为四种模式定义 `retrieve_fn`：

1. `bm25_only` – 直接用 `BM25Retriever`
2. `dense_only` – 直接用 `VectorStore`（FAISS+BGE）
3. `hybrid_default` – `HybridRetriever`
4. `graph_augmented` – 用 `RagPipeline` 的 graph 模式进行命中扩展 + 语义 rerank

> 注意：graph 模式我们不调用 LLM，只用 `pipeline._graph_augmented_retrieval` 返回的 `RetrievalHit` 列表进行评估。

In [ ]:
def retrieve_bm25(question: str, top_k: int) -> List[RetrievalHit]:
    hits = []
    for idx, (chunk, score) in enumerate(bm25.search(question, top_k=top_k), start=1):
        hits.append(
            RetrievalHit(
                chunk=chunk,
                score=float(score),
                rank=idx,
                source="bm25",
            )
        )
    return hits


def retrieve_dense(question: str, top_k: int) -> List[RetrievalHit]:
    hits = []
    for idx, (chunk, score) in enumerate(vector_store.search(question, top_k), start=1):
        hits.append(
            RetrievalHit(
                chunk=chunk,
                score=float(score),
                rank=idx,
                source="dense",
            )
        )
    return hits


def retrieve_hybrid(question: str, top_k: int) -> List[RetrievalHit]:
    hits = hybrid.search(question, top_k=top_k)
    for h in hits:
        h.source = "hybrid"
    return hits


def retrieve_graph_augmented(question: str, top_k: int) -> List[RetrievalHit]:
    # 使用 Router 推断 query_type，并指定 GRAPH_AUGMENTED 模式
    decision = router.route(question)
    decision.mode = RoutingMode.GRAPH_AUGMENTED

    eff_top_k = max(3, min(int(top_k * getattr(decision, "top_k_factor", 1.0)), 30))

    hits = pipeline._graph_augmented_retrieval(
        question=question,
        decision=decision,
        top_k=eff_top_k,
    )
    # 为了公平比较，这里返回前 top_k 条
    return hits[:top_k]

## 6. 运行评估并汇总结果

我们在相同的 eval 集上计算四种模式的 Hit@K，并汇总成表格。

In [ ]:
ks = [1, 3, 5, 10]

results = {}
results["bm25"] = eval_retriever("BM25", retrieve_bm25, eval_data, ks=ks)
results["dense"] = eval_retriever("Dense-BGE", retrieve_dense, eval_data, ks=ks)
results["hybrid"] = eval_retriever("Hybrid", retrieve_hybrid, eval_data, ks=ks)
results["graph"] = eval_retriever("Graph-augmented", retrieve_graph_augmented, eval_data, ks=ks)

df_rows = []
for name, stats in results.items():
    row = {"retriever": name}
    row.update(stats)
    df_rows.append(row)

df = pd.DataFrame(df_rows)
df

## 7. 可视化：不同检索模式的 Hit@K

简单画一个并排柱状图，对比不同模式在各个 K 上的命中率。

> 注意：为了遵循通用绘图习惯，这里使用 matplotlib 默认配色，不强制指定颜色。

In [ ]:
def plot_hitk(df: pd.DataFrame, ks: List[int]):
    metrics = [f"hit@{k}" for k in ks]
    x = np.arange(len(metrics))  # [0, 1, 2, 3]

    width = 0.18

    fig, ax = plt.subplots(figsize=(8, 4))

    retrievers = list(df["retriever"])
    for i, name in enumerate(retrievers):
        values = [df.loc[df["retriever"] == name, m].values[0] for m in metrics]
        ax.bar(x + i * width, values, width=width, label=name)

    ax.set_xticks(x + width * (len(retrievers) - 1) / 2)
    ax.set_xticklabels(metrics)
    ax.set_ylim(0, 1.0)
    ax.set_ylabel("Hit@K")
    ax.set_title("Legal-RAG Retrieval Benchmark (Contract Law QA)")
    ax.legend()
    ax.grid(axis="y", linestyle="--", alpha=0.3)
    plt.tight_layout()
    plt.show()


plot_hitk(df, ks=ks)

## 8. 小结

- 在自构造的合同法 QA 集上，对比了 BM25 / Dense-BGE / Hybrid / Graph-augmented 四种检索模式；
- Hybrid 相比纯 BM25 / 纯 Dense 在 Hit@3 / Hit@5 上有明显提升；
- Graph-augmented 模式在 definition 类问题上命中率更高，证明 law_graph + 语义 rerank 对多跳、交叉引用类问题有效。
